# **13.3 Handling_Specific_Exceptions**

Catching every exception with a single broad `except` is like using a Master Ball on every Pokemon — effective but wasteful. Professional code catches **specific** exception types so each error gets exactly the right response. In this lesson you'll learn to use multiple except clauses, catch related exceptions in groups, and build error-handling logic that is precise, readable, and maintainable.

---

## **Multiple except Clauses**

A single `try` block can have as many `except` clauses as you need, each catching a different exception type. Python checks them in order and runs the first matching one.

In [ ]:
def get_pokemon_level(pokedex: dict, name: str, index: int) -> int:
    """
    Retrieve the level of a Pokemon from the Pokedex by name,
    then access a specific stat by index.
    Each failure mode has its own informative message.
    """
    try:
        stats = pokedex[name]          # May raise KeyError
        level = stats[index]           # May raise IndexError
        return int(level)              # May raise ValueError
    except KeyError:
        print(f"'{name}' is not in the Pokedex")
    except IndexError:
        print(f"Stat index {index} doesn't exist for {name}")
    except ValueError as e:
        print(f"Level data is not a valid integer: {e}")
    return None

pokedex = {
    "Pikachu":  [25, 35, 55, 40, 50, 90],   # [level, hp, atk, def, sp_atk, speed]
    "Charizard": [36, 78, 84, 78, 109, 100],
    "Corrupt":  ["bad_data", 45],
}

print(get_pokemon_level(pokedex, "Pikachu",   0))  # → 25
print(get_pokemon_level(pokedex, "Eevee",     0))  # KeyError
print(get_pokemon_level(pokedex, "Pikachu",  10))  # IndexError
print(get_pokemon_level(pokedex, "Corrupt",   0))  # ValueError

---

## **Catching Multiple Exceptions Together**

When two or more exception types should be handled the same way, group them in a tuple inside a single `except` clause. This avoids duplicating the same handler code.

In [ ]:
def parse_battle_stat(raw):
    """
    Parse a stat value that might come in as a string, None, or a bad number.
    Both ValueError and TypeError get the same friendly message.
    """
    try:
        return int(raw)    # Fails with ValueError (bad str) or TypeError (None)
    except (ValueError, TypeError):
        print(f"Could not parse stat '{raw}' — using default of 0")
        return 0

test_values = ["55", "abc", None, 3.7, "  36  "]

for v in test_values:
    result = parse_battle_stat(v)
    print(f"  Input: {v!r:10} → {result}")

---

## **The Exception Hierarchy**

Python's exceptions form a tree — more specific exceptions are subclasses of broader ones. When you catch a parent exception type, you also catch all of its children. Understanding this hierarchy helps you choose the right level of specificity.

In [ ]:
# Key part of Python's exception hierarchy:
#
# BaseException
# └── Exception              ← catch all normal errors with this
#     ├── ArithmeticError
#     │   └── ZeroDivisionError
#     ├── LookupError         ← catches BOTH of the two below
#     │   ├── IndexError
#     │   └── KeyError
#     ├── ValueError
#     ├── TypeError
#     ├── AttributeError
#     ├── OSError
#     │   └── FileNotFoundError
#     └── RuntimeError

# Demonstration: catching the PARENT catches the children too
collections = [
    (["Pikachu", "Charizard"], 5),         # IndexError (child of LookupError)
    ({"name": "Pikachu"}, "hp"),           # KeyError  (child of LookupError)
]

for obj, key in collections:
    try:
        print(obj[key])
    except LookupError as e:
        # Catches BOTH IndexError and KeyError because both inherit from LookupError
        print(f"Lookup failed ({type(e).__name__}): {e}")

---

## **Order Matters: Specific Before General**

Always place more specific exception types before broader ones. If you put `Exception` first, it will catch everything and the specific handlers below it will never run.

In [ ]:
# WRONG ORDER — the specific clauses are unreachable!
def bad_order(data, key):
    try:
        return data[key]
    except Exception:    # ← catches everything — KeyError never reaches below
        print("Some error")
    except KeyError:     # ← NEVER reached
        print("Key not found")

# CORRECT ORDER — specific first, general last
def good_order(data, key):
    try:
        return int(data[key])       # Can raise KeyError OR ValueError
    except KeyError:
        print(f"Key '{key}' missing")    # Specific — checked first
    except ValueError:
        print(f"Value at '{key}' is not numeric")  # Also specific
    except Exception as e:
        print(f"Unexpected error: {e}")  # General — last resort fallback

pikachu = {"level": "25", "hp": "not_a_number"}

print("Level:",  good_order(pikachu, "level"))
print("HP:",     good_order(pikachu, "hp"))
print("Speed:",  good_order(pikachu, "speed"))

---

## **Inspecting the Exception Object**

The exception object `e` carries useful information: its type name, its message, and sometimes additional attributes. This is valuable for logging and for giving users more helpful error messages.

In [ ]:
def describe_exception(e: Exception) -> dict:
    """Extract useful info from any exception for logging."""
    return {
        'type': type(e).__name__,    # e.g. 'ValueError'
        'message': str(e),           # The error message
        'module': type(e).__module__ # Which module raised it
    }

test_cases = [
    lambda: int("Pikachu"),               # ValueError
    lambda: [1,2,3][10],                  # IndexError
    lambda: {"name": "Pika"}["level"],   # KeyError
    lambda: None + 1,                     # TypeError
]

for func in test_cases:
    try:
        func()
    except Exception as e:
        info = describe_exception(e)
        print(f"  {info['type']}: {info['message']}")

---

## **Practical Pokemon Example**

In [ ]:
# A robust Pokemon team validator that gives specific feedback
def validate_team(team_data):
    """
    Validate a list of Pokemon dictionaries.
    Each specific error type gets a different, informative message.
    """
    errors = []
    valid = []
    
    for i, raw in enumerate(team_data):
        try:
            # Validate required fields
            name = raw['name']          # KeyError if missing
            level = int(raw['level'])   # ValueError if not numeric
            
            if not isinstance(name, str):
                raise TypeError(f"name must be str, got {type(name).__name__}")
            
            if not (1 <= level <= 100):
                raise ValueError(f"level {level} out of range 1–100")
            
            valid.append({'name': name, 'level': level})
        
        except KeyError as e:
            errors.append(f"Slot {i}: missing field {e}")
        except ValueError as e:
            errors.append(f"Slot {i}: bad value — {e}")
        except TypeError as e:
            errors.append(f"Slot {i}: wrong type — {e}")
    
    return valid, errors

team_input = [
    {"name": "Pikachu",  "level": "25"},        # Valid
    {"name": "Charizard"},                        # Missing 'level'
    {"name": "Blastoise", "level": "thirty-six"}, # Bad value
    {"name": "Venusaur",  "level": "150"},        # Out of range
    {"name": "Gengar",    "level": "36"},          # Valid
]

valid_pokemon, validation_errors = validate_team(team_input)

print("Valid Pokemon:")
for p in valid_pokemon:
    print(f"  ✓ {p['name']} Lv.{p['level']}")

print("\nValidation errors:")
for err in validation_errors:
    print(f"  ✗ {err}")

---

## **Practice Exercises**

### **Task 1: Two Separate Handlers**

Catch `KeyError` and `ValueError` with separate except clauses, each with its own message.

**Expected Output:**
```
KeyError: 'speed' not found
ValueError: 'fast' is not numeric
```

In [ ]:
pikachu = {"level": "25", "attack": "fast"}

# Test 1: access missing key
# Test 2: convert 'fast' to int

# Your code here:


### **Task 2: Group Same-Response Errors**

Handle `ValueError` and `TypeError` with one `except` clause.

**Expected Output:**
```
25
Bad input — using 0
Bad input — using 0
```

In [ ]:
inputs = ["25", "Pikachu", None]

# Your code here:


### **Task 3: Specific Before General**

Write handlers in the correct order: `FileNotFoundError` first, then `OSError`, then `Exception`.

**Expected Output:**
```
File 'save.txt' not found
```

In [ ]:
# Your code here:


### **Task 4: Exception Type Name**

Catch `Exception` and print the type name and message.

**Expected Output:**
```
ValueError: invalid literal for int() with base 10: 'Charizard'
```

In [ ]:
# Your code here:


### **Task 5: LookupError Parent**

Catch `LookupError` to handle both a `KeyError` and an `IndexError` in the same block.

**Expected Output:**
```
KeyError caught via LookupError
IndexError caught via LookupError
```

In [ ]:
# Your code here:


### **Task 6: Three Specific Handlers**

Handle `KeyError`, `IndexError`, and `ValueError` each with unique messages.

**Expected Output:**
```
Key missing: 'hp'
Index out of range: 99
Bad number: 'forty'
```

In [ ]:
# Your code here (demonstrate each error type):


### **Task 7: Validate Input**

Write `validate_level(raw)` that raises `ValueError` with a clear message for out-of-range or non-numeric input.

**Expected Output:**
```
25 is valid
Error: 'Pikachu' is not a valid integer
Error: 0 is out of range (must be 1–100)
```

In [ ]:
# Your code here:


### **Task 8: Collect Errors**

Validate a list of Pokemon dicts, collecting specific error messages for each bad entry.

**Expected Output:**
```
Valid: ['Pikachu', 'Gengar']
Errors: ['Slot 1: missing level', 'Slot 2: bad value thirty']
```

In [ ]:
team = [
    {"name": "Pikachu",  "level": "25"},
    {"name": "Blastoise"},
    {"name": "Venusaur",  "level": "thirty"},
    {"name": "Gengar",    "level": "36"},
]

# Your code here:


### **Task 9: ArithmeticError**

Catch `ArithmeticError` (parent of ZeroDivisionError) to handle division by zero.

**Expected Output:**
```
Arithmetic error: division by zero
```

In [ ]:
# Your code here:


### **Task 10: Fallback Handler**

Write a function with specific handlers for known errors, plus a final `except Exception` fallback.

**Expected Output:**
```
Known: KeyError
Known: ValueError
Unexpected: TypeError (unsupported operand type(s))
```

In [ ]:
# Your code here:


---

## **Summary**

- Multiple `except` clauses let each error type get its own response
- Group related errors with `except (TypeError, ValueError):`
- Always put **specific** handlers before **general** ones
- Catching a parent catches all its children (e.g. `LookupError` catches `KeyError` + `IndexError`)
- `type(e).__name__` gives you the error's class name as a string
- End with `except Exception` as a final fallback for unexpected errors
- Never use `except Exception` as your *only* handler for known errors

---

## **Quick Reference**

```python
# Multiple specific handlers
try:
    ...
except KeyError:
    ...
except ValueError:
    ...
except Exception as e:
    print(f"Unexpected: {type(e).__name__}: {e}")

# Group same-response errors
except (ValueError, TypeError) as e:
    print(f"Bad input: {e}")

# Exception hierarchy reminders
# LookupError → KeyError, IndexError
# ArithmeticError → ZeroDivisionError
# OSError → FileNotFoundError
# Exception → all of the above

# Inspect the exception
type(e).__name__   # 'ValueError'
str(e)             # error message
```